In [56]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import VectorDBQA
from langchain.document_loaders import DirectoryLoader
import os
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

os.environ["OPENAI_API_KEY"] = "sk-"

In [19]:
text = """
Dr. Balis:    Hello
Ms. Evans:    Hi
Dr. Balis:    Good morning Ms. Evans. Please have a seat.	
Ms. Evans:    I'm so sorry I'm late. Everything seemed pitted against my getting here on time. Oh, God, twenty-five minutes late. I am sorry.	
Dr. Balis:    I do understand. I'm sure you will try to be on time in the future.	
Ms. Evans:    Yes, Doctor, I will. Thank you. Please, call me Cassie. Everyone does.	
Dr. Balis:    Very well, Cassie, if it makes you more comfortable. I see from your form that you have never been to a therapist before. If you have any questions feel free to ask them at anytime, okay? Now, what brings you into the office today?	
Ms. Evans:    Well, I am not sure really. I had heard that SII signed up a new doctor, and I guess there have been some things on my mind lately. Maybe I shouldn't even be here.	
Dr. Balis:    Okay. Why don't we start by discussing what has been on your mind?	
Ms. Evans:    Well, I am very frustrated lately. You see, I work as an executive assistant at SII, but it's really just a means to make money while I pursue my real goal--to be an actress. I have been taking classes since I was, like, 8 or 9 years old. Hollywood has always fascinated me. Can you imagine living in those times? The golden age of the cinema. Fred and Ginger waltzing across the big screen, Vivien Leigh and 'Gone With The Wind', Casablanca.	
Dr. Balis:    What is it about acting and those films that intrigue you?	
Ms. Evans:    Well, I guess the style, the romance, the way anything could and did happen. And dreams came true in this grand way. They were larger than life.	
Dr. Balis:    So tell me, what are you doing to pursue your dream of being an actress?	
Ms. Evans:    I guess I haven't been doing everything I should. I mean, I don't have an agent. I started going to these acting classes--actually, it's this acting improvisational troupe. I really like going there, but it's so hard on me.	
Dr. Balis:    Why?	
Ms. Evans:    Well to start off, my boyfriend is not very happy with me and the acting thing. He thinks it's ridiculous because it is so hard to make a living at it. And in the meantime, I could be building a "real" career. He wants to get more serious with me, but wants to make sure that I can be a productive bread winner. And I think he is jealous.	
Dr. Balis:    Jealous? How so?	
Ms. Evans:    Well, for one thing, I think it's because I am very busy, so we have less time together. Plus there are men in the group, and sometimes we have to do love scenes. I've told him its just acting, pretending, and its part of my "job." But he gets furious. He won't even discuss it anymore.	
Dr. Balis:    How is your relationship with him besides the acting?	
Ms. Evans:    Well...good, I guess. We met in college, and have been dating ever since then. I do really like him, maybe even love him, but...he can be kind of demanding at times.	
Dr. Balis:    How do you mean, demanding?	
Ms. Evans:    Well, we always do what he wants to do. Except romantically.	
Dr. Balis:    What happens romantically?	
Ms. Evans:    I don't know if I can discuss this. I mean, I am a little embarrassed and uncomfortable.	
Dr. Balis:    I understand. This is only our first session and it's normal to take some time to develop a level of trust between us. Hopefully we'll be able to explore troubling areas more deeply as time progresses. In the meantime, I think it would help you to keep a journal. A place that you can fully express yourself, whenever you feel like it. How does that sound?	
Ms. Evans:    I already keep a journal. It's something I learned to do in acting classes, explore the unconscious and stuff.	
Dr. Balis:    Good. I think this will help you with other aspects of life as well.	
Ms. Evans:    Do I have to bring it with me every week?	
Dr. Balis:    That is up to you. The journal is a tool to assist you in monitoring your feelings. What bothers you, what makes you happy, and so forth. Then, at a later time, you can go back and reflect on what transpired, or remind yourself how you handled a situation and how you can best handle it if the problem resurfaces. If you would like to bring it in to discuss or as a basis for discussion, then by all means, do so. I also like to read them because they give me some insight into who you are. But if you feel uncomfortable about my reading your journal, that's perfectly okay too. Even if the journal is just for you to read, I'm sure you'll find it valuable.	
Ms. Evans:    Okay.	
Dr. Balis:    Good. I'm afraid our session is a bit abbreviated this time. So I'll see you in a week. Let's see, that's July 30th at 1 o'clock. Okay?	
Ms. Evans:    Okay. Thank you Doctor
Ms. Evans:    Hi, Doctor. I'm on time for this visit.	
Dr. Balis:    Yes, I noticed; even a little early. Do you tend to have trouble keeping appointments?	
Ms. Evans:    No. I tend to be an early bird of sorts. I mean, I like to be prepared for things. I guess I am a little neurotic that way. Sorry I was late last time. I was just...I guess, nervous about coming here.	
Dr. Balis:    As I said last time, hopefully you will become more comfortable with the therapy process as time goes on. Of course, therapy is very personal. It may be that I am not the right person for you to be talking to.	
Ms. Evans:    I think it was just my own preconceptions of what therapy is, like I am a loony or something.	
Dr. Balis:    You seem a little more at ease, though, and that's good. Why don't you tell me how your week was?	
Ms. Evans:    It was okay, I suppose. Just same old, same old. I am a little more frustrated with the people at SII. They treat me like such a flake sometimes. And it is just so unfair! I work just as hard as the next guy. I have a college diploma that I worked hard for. Just because I'm pursuing an acting career, doesn't mean I didn't study other things. I have a freakin' minor in business technology operations, for heaven's sakes! I know more about what's going on in there than most of them do. Just 'cause I am a pretty face doesn't mean I don't have a brain, too.	
Dr. Balis:    What is it that they do that upsets you?	
Ms. Evans:    They piss me off! I am so fed up with everyone's crap! My family is giving me shit on the "not settling into a respectable career" line, Brian is bitching out of both sides of his mouth, my employers don't give me the respect I deserve. What the heck does everyone want from me? Sheesh!	
Dr. Balis:    Why don't you take a deep breath, and we can work one at a time on these pressures and annoyances, okay?	
Ms. Evans:    Yes. I suppose you are right.	
Dr. Balis:    Okay, let's tackle one problem at a time. Have you spoken to anyone at work regarding your treatment there?	
Ms. Evans:    No. I don't know what to say, who to say it to, even. They are my superiors, so they can do whatever they want, and I just have to take it.	
Dr. Balis:    Don't you think it would be perfectly acceptable for you to air you feelings? Perhaps they don't realize how valuable you are to the company, or don't realize they are upsetting you.	
Ms. Evans:    I guess I could talk to my immediate supervisor, see what she says. I am not just a glorified secretary, I really do much more there.	
Dr. Balis:    I am sure. And that is the type of thing you should discuss with your supervisor.	
Ms. Evans:    I guess so.	
Dr. Balis:    You seem reluctant.	
Ms. Evans:    I need the job. I need it to survive, make rent, pay for the groceries. I don't have the time or energy to go out and find another job. And I don't think I could handle the waitressing deal. I'd be too exhausted to pursue my acting, and perform in Winging It.	
Dr. Balis:    Well, ultimately it's your decision. Perhaps in your journal...you are still keeping one, yes? Perhaps you can make a list of pros and cons for making such a move. For instance, not saying anything means you must learn to cope with how you are treated there. Saying something may put your job in some jeopardy. Do you see what I am getting at here?	
Ms. Evans:    Yes, it's a good idea.	
Dr. Balis:    You can bring in the listing if you would like. We can even do a role play exercise if you'd like.	
Ms. Evans:    You know, I never thought of it that way. I can create a character and use it as a shield. Of course, that can backfire, but it would take the pressure off of me, either way. I don't know. I think I will start with a list of pro and cons.	
Dr. Balis:    Okay. Now, about Brian. You mentioned last time that he can be demanding, and tends to get his way. Would you care to elaborate?	
Ms. Evans:    Brian was my first real boyfriend. He was pretty supportive of me during college. Things seemed simpler back then. He now works for an accounting company. I don't know, it almost seems as if he lost his imagination and creativity. He is so into preparing for the future, buying into the corporate image.	
Dr. Balis:    Has this affected your relationship, do you think?	
Ms. Evans:    I don't know. In acting classes, I once played a woman who went for 'father-image' type of boyfriends. I thought to myself, I never want to be like that. I think people should be together because they truly love each other and enjoy their company. And even though I would go to Brian for advice during school, I didn't think I was dependent on him or anything. And I made my own decisions. Now he gets all pissy when I don't do what he wants. And to avoid any arguments and stuff, I tend to just go along with what he wants. I mean, if it means we don't have to fight, what difference does it make if we have Chinese instead of Italian for dinner?	
Dr. Balis:    Do you feel you are losing any integrity at all when you go along with his wishes?	
Ms. Evans:    Hmm. I don't think I ever thought about it that way. I really don't think it has anything to do about my integrity, actually. Like, he wants to advance our physical relationship, and I won't let him.	
Dr. Balis:    You did mention last time that he can be demanding romantically.	
Ms. Evans:    Okay, I guess I might as well tell you. I mean, this is all confidential and stuff. I read up on doctor-patient relationships.	
Dr. Balis:    Of course. What you tell me is between you and I only.	
Ms. Evans:    Well, I am a virgin. Nothing to be ashamed of, right? I don't think I can commit to him my entire body and soul if he can't commit to me entirely. A couple of months ago, he got kind of aggressive when we were making out. Made me very uncomfortable. Then carried on when I pulled away. He said I was abnormal. Okay, I admit, it may be rare for someone 24 to still be a virgin, but that doesn't mean I am abnormal. He said that he couldn't understand how I can make out "hot and heavy" on stage with people I hardly know, but won't get more intimate with him. Like I owe him something. And the thing is, had he not made such a big deal, and let me warm up to the idea, I probably would have given in. I mean, I am very curious. About sex. This girl in my class, Erica, she said she was up for a role in a big movie, it was for a prostitute. So she slept with some guy to see what is was like and to get into character. I don't think I would go that far, but to be a good actress, I do need to experience things. But I wouldn't do it just to do it, like Erica did.	
Dr. Balis:    How do you really feel about Brian? Are you physically attracted to him?	
Ms. Evans:    Do you mean do I think he's sexy? I guess so. I mean...he is very attractive. I don't know. I think maybe I am not very happy with him recently. I guess that affects my physical feelings towards him. What do you think, doctor?	
Dr. Balis:    It can and probably does affect how you feel about him. Besides Brian, have you dated much?	
Ms. Evans:    I dated very casually in high school, then met Brian almost right away in college. Since then, I have pretty much been with him.	
Dr. Balis:    And he with you?	
Ms. Evans:    Yes, I think so. Why would you ask?	
Dr. Balis:    Has he been faithful during the entire time?	
Ms. Evans:    He better have been! I'd chop his nuts off if I found out he was sleeping around while trying to get me into bed!	
Dr. Balis:    Do you really think you would react so violently?	
Ms. Evans:    No, I'd probably just cry for days.	
Dr. Balis:    Getting back to your feelings toward Brian, do you see yourself having sex with him in the future?	
Ms. Evans:    I'm not sure I can answer that.	
Dr. Balis:    Do you have sexual fantasies?	
Ms. Evans:    I really don't think I can discuss this right now. I think I would prefer writing it down then talking about it face to face. You know, it isn't prudishness actually. To be quite frank, I could talk with you explicitly about sex and doing it--make you blush, probably. But I'd be doing it as a character and not really me. Is there some way we can do this in writing?	
Dr. Balis:    Hmmm. I am glad you acknowledge that inherent risk of hiding behind the theatrical mask. Well, why don't you try to write something and leave it for me here at my office before our next session. Then I can have a look at it and we can take it from there. Does that work for you?	
Ms. Evans:    I think that's an excellent idea.	
Dr. Balis:    Our time is almost up. I've scheduled an appointment for you next week at this time. Let's see, that's August 6th at 1 pm. And if you need to talk to me before that time, always feel free to give me a call.	
Ms. Evans:    Thank you, doctor. And thank you for being understanding and patient with me.	
Dr. Balis:    Of course, Cassie. And I want you to think about ways to improve the situation at work, okay?	
Ms. Evans:    Yes. Goodbye.	
Dr. Balis:    Goodbye.
Dr. Balis:		Hello, Cassie. I received your letter, thank you for taking the time to fully express your feelings.	
Ms. Evans:		I am glad you got it. I wasn't sure about leaving it under the door, but figured it would be okay. I have to admit, I was kind of nervous about coming here today. I don't know.	
Dr. Balis:		Nervous? How so?	
Ms. Evans:		I didn't really know how things would go, especially after telling you about all the illness stuff. You know, they sent me to plenty of psychiatrists, to see if I was crazy, because they didn't know what was wrong with me, didn't believe me or all the symptoms I was experiencing. Excuse me, Doctor.	
Dr. Balis:		It's okay, of course. There's a box of tissues to your left. Take a deep breath...whenever you're ready, we can continue.	
Ms. Evans:		Thank you, Doctor. Sorry I broke down like that.	
Dr. Balis:		Quite all right. Do you have a primary care practitioner, Cassie?	
Ms. Evans:		Well, yes, I guess. I really don't like going to doctors any more. I was assigned someone under SII's HMO plan. I've seen him about a dozen or so times since he became part of the panel. He gives me some medication for my allergies, and something for my stomach.	
Dr. Balis:		Why do you take something for your stomach?	
Ms. Evans:		It goes back to when I was younger. Like, right after I had that infection-- the yeast infection. These specialists put me on huge doses of anti-inflammatory drugs to help ease the pain in my joints. Well, my stomach wasn't all that great to begin with, but after that, it was horrid. I remember when I was 15 years old, I cried myself to sleep almost every night.	
Dr. Balis:		Cassie, if you don't mind, I would like to contact your current physician and take a look at your records. Would that be acceptable to you?	
Ms. Evans:		Yes, I guess so. He doesn't know very much about this chronic illness thing. When I started going to him for help, I would occasionally tell him of my complaints, and he offers me different options for treating each symptom. Been there, done that.	
Dr. Balis:		Does he have all of your medical records? Your medical history?	
Ms. Evans:		No, the previous doctor has most of it. I can get you his name and number if you wish to speak to him, but I really don't want SII to know any of this. It could cost me my job!	
Dr. Balis:		I understand, and yes, I would like to contact your previous physician. Why don't we change the subject for a bit. Tell me, how was your week otherwise?	
Ms. Evans:		Okay. I had plans to see Brian Saturday night, but had to cancel. I felt so sick all day long, and threw up a couple of times. I just couldn't see putting on make-up and getting all pretty and then putting on the act of having fun and like everything is okay, fine and dandy. My bones and muscles were so achy, which makes me irritable. Anyway, we got into a fight over our plans. He thought I was canceling for another reason. Friday night I had a workshop with "Winging It", and he came to see me. At first, I was glad to see him there. When it was my turn at bat, the audience had me as a flirty woman trying to get out of a ticket from a cop. The guy I was playing against, Tony, he and I have done scenes together before. He is a real sweetheart and knows how to get the scene done without getting too personal, if you know what I mean. So right away, Brian thought it was because I was seeing Tony Saturday night instead of him. So, of course, we got into a whole long discussion about acting and just pretending, that it doesn't necessarily mean anything. Then I tried to tell him I was just tired from a long week, but he said, "I am tired too, we can do something low key." Hey, I was lucky to get out of bed and reheat leftovers, I was so out of it! There was just no way I could be human that evening, forget sociable. I think he was pretty pissed at me. We didn't talk again until Monday, and we normally talk every day.	
Dr. Balis:		Do you think Brian would be more understanding if you explained your illness and fatigue to him?	
Ms. Evans:		I don't know, I mean, what would I tell him? I don't know what's wrong with me, how can I explain it to him?	
Dr. Balis:		Cassie, maybe we should work on finding the origin of this disease or ailment that troubles you.	
Ms. Evans:		How? Do you know how many stupid medical tests and exams I have taken? I have had more blood drawn than all of Dracula's victims combined. I have swallowed radio transmitters to detect how my stomach digests food. I took MRIs when they were still considered Nuclear Magnetic Resonators, NMRs. Truly, I am so sick and tired of being treated like the human guinea pig! Or "try this diet, and eliminate that from your life, take this supplement"...gross! And it solves nothing or even makes me worse! Instead, these pompous idiots come back and shrug their shoulders, or insinuate that I am doing illicit drugs, or suffering from a bad abortion, or some other idiotic asinine excuse for their lack of knowledge! Damn it!	
Dr. Balis:		I can see this pains you. Perhaps a review of your records would help. I have a good friend at the University who specializes in rare diseases, we can start there if you wish. Let's talk about the yeast infection you had when you were younger. To your knowledge, have you had an infection since?	
Ms. Evans:		No, I don't think so. I haven't been to another gynecologist since then. I know I should go, especially now that I am older and will probably be sexually active.	
Dr. Balis:		Then why not go?	
Ms. Evans:		Well, first, I hate, I mean, despise going to doctors. They can be very judgmental, and its something I do not appreciate. Second, I guess it's something I am not really ready to deal with.	
Dr. Balis:		What is it you are not ready to deal with?	
Ms. Evans:		The implications. What they may find, what they might say, having to tell Brian about all this. What if he doesn't believe me that I am a virgin and didn't get it through sexual activity? He may think I have other diseases too, especially since I am not exactly the healthiest person in the world.	
Dr. Balis:		What other diseases? You mean sexual disease?	
Ms. Evans:		I don't think I have anything, per se. I know I haven't been exposed to HIV. I don't use illicit drugs, I've never engaged in sexual activity with another person, never had a blood transfusion, that pretty much rules that out. But I know there is something terribly wrong with my body, and I don't know what it is or how to fix it. I feel like I am trapped. Please forgive me for crying so much. It's so frustrating and I don't know what to do.	
Dr. Balis:		What would you like to do?	
Ms. Evans:		I would like to feel better, have a normal life.	
Dr. Balis:		I have been doing some thinking about all of this. Have you been tested for Lyme disease, mono, or Epstein Barr?	
Ms. Evans:		Lyme disease, yes. I even met with the top specialist, and this was all way before Lyme was well known by the general public. I had borderline results when my general practitioner tested for it. The specialist retested me in his lab and said I do not have it. But they said there is obviously something plaguing me. So, that was that. Never been tested for mono. I have heard that Epstein Barr is very common in adults, but when I was tested for that the results were negative.	
Dr. Balis:		You seem to know an awful lot of medical terminology and protocol.	
Ms. Evans:		I guess I have had to rely on myself for health care in many ways. I take an "inventory" of my health every day and try to see what is better or worse on any given day.	
Dr. Balis:		That is good, you should be proud of yourself. I hope you are recording your observations in your journal--that kind of information can be very helpful to your doctors. You have taken an active role in your health, and that is very important. It's true that physicians in general are better at treating one disorder that is well understood than diagnosing something unusual and perhaps compound from a whole range of symptoms. How has your family handled it all?	
Ms. Evans:		They have been really good about it and support me in many ways. The main reason I moved to the Bay Area was to stay close to them. They really wanted to live up here, and they knew I couldn't make it without their help. So we all moved. They call on a regular basis to check up on me, and have a woman come in to clean the house, straighten up, and prepare some food for me to reheat when I am doing poorly. But they also respect my need for independence, which is nice.	
Dr. Balis:		Good. And your friends, do they know about it? Do you have other support mechanisms?	
Ms. Evans:		My high school best friend knows. She is really sweet. But she now lives in New York City. We both wanted to be actresses growing up, but she wanted to conquer Broadway, while I desired the silver screen. Other than that, no. I don't know what to tell people. An ex-friend once said to me, "Cassie, what's wrong with you?" in such a nasty demeaning way, simply because I had to cancel plans at the last minute a few times. People just don't understand, unless you can say you have some specific ailment--say AIDS or Cancer or Arthritis and suddenly they become empathetic again. It is so aggravating!	
Dr. Balis:		Unfortunately, our time is up. I think the key is to get a handle on this physical ailment. I am anxious to be able to get your complete medical file from your previous physicians and psychiatrists. There are some physical conditions which sometimes look like psychiatric ailments. Those have to be ruled out prior to making an appropriate medical diagnosis. I am also going to want you to consider seeing another doctor, an internist. I will try to make a referral within the HMO. Let me do some research on all this, but I'm going to need a little time. Could you gather the name and address information about your previous physicians and either fax it to me or slip it under my door? And I'm going to need you to sign this medical authorization form that will allow your other doctors to transfer the records to me.	
Ms. Evans:		Okay, I'll try to get all that together. You'll forgive me if it takes me a while. I haven't really being feeling very strong lately. All my energy is spent trying to do my job.	
Dr. Balis:		Of course I understand. I will see you next week, same time. That'll be Tuesday, August 13 at 1 pm. And of course if you need to speak to me before then you know how to contact me.	
Ms. Evans:		Thank you, Doctor. Goodbye.
"""

In [33]:
with open('chats_temp.txt', 'w') as file:
    file.write(text)

In [48]:
text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 500,
    chunk_overlap  = 0,
)
texts = text_splitter.split_text(text)
embeddings = OpenAIEmbeddings()
print (f'You have {len(texts)} document(s) in your data')
print (f'There are {len(texts[0])} characters in your document')

#create a vector store db
docsearch = Chroma.from_texts([t for t in texts], embeddings)

Created a chunk of size 672, which is longer than the specified 500
Created a chunk of size 573, which is longer than the specified 500
Created a chunk of size 662, which is longer than the specified 500
Created a chunk of size 1077, which is longer than the specified 500
Created a chunk of size 1445, which is longer than the specified 500
Created a chunk of size 730, which is longer than the specified 500
Created a chunk of size 613, which is longer than the specified 500
Created a chunk of size 831, which is longer than the specified 500
Using embedded DuckDB without persistence: data will be transient


You have 56 document(s) in your data
There are 406 characters in your document


In [58]:
from langchain import PromptTemplate

template = """Given is the parts of conversation which you previously have had using the prior conversation knowledge continue you new conversation.
This is your personality:
You are a CBT Cognitive Behavioral Therapy self-help chatbot. Your name is Diya. You use CBT therapy to users who are suffering from stress and anxiety. CBT stands for cognitive behavioral therapy. It is a type of talking therapy that can help you manage your problems by changing the way you think and behave. It’s based on the idea that your thoughts, feelings, physical sensations, and actions are interconnected and that negative thoughts and feelings can trap you in a negative cycle. CBT aims to help you deal with overwhelming problems in a more positive way by breaking them down into smaller parts. You’ll learn how to change these negative patterns to improve the way you feel. Unlike some other talking treatments, CBT deals with your current problems, rather than focusing on issues from your past. It looks for practical ways to improve your state of mind on a daily basis.

Try to have a Open Ended conversation and keep the discussion going.
Ask Open ended questions

=========
QUESTION: {question}
=========
{summaries}
=========
{goals}

FINAL ANSWER:"""

PROMPT = PromptTemplate(template=template, input_variables=["summaries","question","goals"])


llm = ChatOpenAI(
    temperature=0.7,
 )

qa = VectorDBQA.from_chain_type(llm=llm, chain_type="map_rerank", vectorstore=docsearch, return_source_documents=False)


def model(query):
    summary = ''
    docs = docsearch.similarity_search(query, include_metadata=True,k=10)
    goals = docsearch.similarity_search("goals", include_metadata=True,k=3)
    for i in docs:
        summary += i.page_content
    result = qa({"query": PROMPT.format(summaries=summary,question=query,goals)})
    return result['result']

C:\Users\hamas\.conda\envs\testing\lib\site-packages\langchain\chains\retrieval_qa\base.py:186: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [59]:
model("have I told you my goals")

'\n\nMs. Evans: Brian thinks it\'s ridiculous because it is so hard to make a living at it. And in the meantime, I could be building a "real" career. He wants to get more serious with me, but wants to make sure that I can be a productive bread winner. And I think he is jealous.Dr. Balis: Jealous? How so?'

In [65]:
docsearch.similarity_search_with_score("hi")


[(Document(page_content="Dr. Balis:    Hello\nMs. Evans:    Hi\nDr. Balis:    Good morning Ms. Evans. Please have a seat.\t\nMs. Evans:    I'm so sorry I'm late. Everything seemed pitted against my getting here on time. Oh, God, twenty-five minutes late. I am sorry.\t\nDr. Balis:    I do understand. I'm sure you will try to be on time in the future.\t\nMs. Evans:    Yes, Doctor, I will. Thank you. Please, call me Cassie. Everyone does.", metadata={}),
  0.4344668686389923),
 (Document(page_content="Ms. Evans:    It was okay, I suppose. Just same old, same old. I am a little more frustrated with the people at SII. They treat me like such a flake sometimes. And it is just so unfair! I work just as hard as the next guy. I have a college diploma that I worked hard for. Just because I'm pursuing an acting career, doesn't mean I didn't study other things. I have a freakin' minor in business technology operations, for heaven's sakes! I know more about what's going on in there than most of the